<a href="https://colab.research.google.com/github/fivetwentythree/colab-notebooks/blob/main/invoice_sort_out.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code works with a some heuristics, here is an explanation on what is happening here

How the Heuristic Works
Amount Match:

First, filter transactions with the same Amount as the invoice.
Date Tolerance:

Among amount matches, select transactions where the Transaction Date is within a specified range (e.g., ±3 days of the Due Date).
You can adjust the range via the date_tolerance_days parameter.
Mark Invoice as Paid:

If a match is found, update the invoice as Paid and link the transaction ID.

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Load invoice data
invoice_file = "invoices.csv"
invoices = pd.read_csv(invoice_file)

# Load bank transactions
transaction_file = "transactions.csv"
transactions = pd.read_csv(transaction_file)

# Convert necessary columns to consistent formats
invoices['Amount'] = invoices['Amount'].astype(float)
transactions['Amount'] = transactions['Amount'].astype(float)
invoices['Due Date'] = pd.to_datetime(invoices['Due Date'])
transactions['Date'] = pd.to_datetime(transactions['Date'])

# Match invoices to transactions
def reconcile_with_heuristic(invoices, transactions, date_tolerance_days=3):
    # Add a Paid Status column to invoices if it doesn't exist
    if 'Paid Status' not in invoices.columns:
        invoices['Paid Status'] = 'Unpaid'

    # Add a Matched Transaction column for clarity
    invoices['Matched Transaction'] = None

    for idx, invoice in invoices.iterrows():
        # Find transactions matching by amount
        amount_matches = transactions[transactions['Amount'] == invoice['Amount']]

        if not amount_matches.empty:
            # Narrow matches by date tolerance
            for _, match in amount_matches.iterrows():
                transaction_date = match['Date']
                due_date = invoice['Due Date']

                # Check if the transaction date is within the tolerance range of the due date
                if abs((transaction_date - due_date).days) <= date_tolerance_days:
                    # Mark the invoice as paid
                    invoices.at[idx, 'Paid Status'] = 'Paid'
                    invoices.at[idx, 'Matched Transaction'] = match['Transaction ID']

                    # Remove the matched transaction from the pool to avoid duplicate matches
                    transactions = transactions[transactions['Transaction ID'] != match['Transaction ID']]
                    break

    return invoices

# Reconcile invoices with heuristic
updated_invoices = reconcile_with_heuristic(invoices, transactions)

# Save the updated invoices to a new file
output_file = "reconciled_invoices_with_heuristic.csv"
updated_invoices.to_csv(output_file, index=False)

print(f"Reconciliation complete with heuristic. Updated invoice file saved to {output_file}.")


This is another alternative script which uses the fuzzywuzzy to further match the fields for better efficient sorting.

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from datetime import datetime, timedelta

# Load invoice data
invoice_file = "invoices.csv"
invoices = pd.read_csv(invoice_file)

# Load bank transactions
transaction_file = "transactions.csv"
transactions = pd.read_csv(transaction_file)

# Convert necessary columns to consistent formats
invoices['Amount'] = invoices['Amount'].astype(float)
transactions['Amount'] = transactions['Amount'].astype(float)
invoices['Due Date'] = pd.to_datetime(invoices['Due Date'])
transactions['Date'] = pd.to_datetime(transactions['Date'])

# Fuzzy Matching Threshold
FUZZY_THRESHOLD = 80

# Match invoices to transactions with fuzzy matching and heuristic
def reconcile_with_fuzzy_matching(invoices, transactions, date_tolerance_days=3):
    # Add a Paid Status column to invoices if it doesn't exist
    if 'Paid Status' not in invoices.columns:
        invoices['Paid Status'] = 'Unpaid'

    # Add a Matched Transaction column for clarity
    invoices['Matched Transaction'] = None

    for idx, invoice in invoices.iterrows():
        # Find transactions matching by amount
        amount_matches = transactions[transactions['Amount'] == invoice['Amount']]

        if not amount_matches.empty:
            # Narrow matches by date tolerance
            for _, match in amount_matches.iterrows():
                transaction_date = match['Date']
                due_date = invoice['Due Date']

                # Check if the transaction date is within the tolerance range of the due date
                if abs((transaction_date - due_date).days) <= date_tolerance_days:
                    # Use fuzzy matching on the Description field for customer name or invoice ID
                    description = str(match['Description']).lower()
                    customer_name = str(invoice['Customer']).lower()
                    similarity_score = fuzz.partial_ratio(customer_name, description)

                    if similarity_score >= FUZZY_THRESHOLD:
                        # Mark the invoice as paid
                        invoices.at[idx, 'Paid Status'] = 'Paid'
                        invoices.at[idx, 'Matched Transaction'] = match['Transaction ID']

                        # Remove the matched transaction from the pool to avoid duplicate matches
                        transactions = transactions[transactions['Transaction ID'] != match['Transaction ID']]
                        break

    return invoices

# Reconcile invoices with fuzzy matching
updated_invoices = reconcile_with_fuzzy_matching(invoices, transactions)

# Save the updated invoices to a new file
output_file = "reconciled_invoices_with_fuzzy_matching.csv"
updated_invoices.to_csv(output_file, index=False)

print(f"Reconciliation complete with fuzzy matching. Updated invoice file saved to {output_file}.")


Steps to Test the Enhanced Script
Add Date Columns to Your CSV Files:

For invoices.csv, ensure you include Due Date:

In [ ]:
Invoice ID,Customer,Amount,Due Date
1001,John Doe,200.50,2024-11-30
1002,Jane Smith,150.00,2024-11-30


For transactions.csv, ensure you include Date:

In [ ]:
Transaction ID,Date,Description,Amount
T001,2024-12-01,Payment from John Doe,200.50
T002,2024-11-28,Payment,150.00
